In [2]:
# Force-install a compatible version of NumPy
!pip install "numpy<2.1" --force-reinstall

print("\n" + "="*50)
print("✅ INSTALL COMPLETE!")
print("CRITICAL: Go to 'Kernel' -> 'Restart' in the menu above.")
print("Then continue to the next cell.")
print("="*50)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 11.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5

✅ INSTALL COMPLETE!
CRITICAL: Go to 'Kernel' -> 'Restart' in the menu above.
Then continue to the next cell.


In [2]:
data = np.load('/home/Capstone_Team78/Dataset/Datasets/UnfilteredSubtypeData.npy')
print(f"🔑 Keys found in this file: {data.files}")

🔑 Keys found in this file: ['X', 'y', 'feature_names']


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SpatialDropout1D
import os

# --- 1. Robust Keras 3 Compatibility Wrapper ---
class FixedSpatialDropout1D(SpatialDropout1D):
    def __init__(self, **kwargs):
        # Keras 3 no longer accepts these legacy arguments in __init__ for this layer
        kwargs.pop('trainable', None)
        kwargs.pop('noise_shape', None)
        super().__init__(**kwargs)

# --- 2. Paths ---
DATA_PATH = '/home/Capstone_Team78/Dataset/Datasets/UnfilteredSubtypeData.npy'
MODEL_PATH = '/home/Capstone_Team78/Dataset/Models/thyroid-subtype-optimized.h5'

# --- 3. Reload Data ---
print("📂 Loading Genomic Data into RAM...")
archive = np.load(DATA_PATH, allow_pickle=True)
X_train_sub = archive['X'].astype('float32') 
y_train_sub = archive['y']
cpg_names = archive['feature_names'].tolist()

# --- 4. Reload Model with Custom Object Mapping ---
print("🤖 Loading Optimized Model (Stripping legacy Keras arguments)...")
custom_objects = {'SpatialDropout1D': FixedSpatialDropout1D}

# loading the 92% accuracy model for SHAP
final_model_v2 = tf.keras.models.load_model(
    MODEL_PATH, 
    custom_objects=custom_objects, 
    compile=False
)

print(f"✅ Success! Loaded {X_train_sub.shape[1]} features and stabilized the model.")

📂 Loading Genomic Data into RAM...
🤖 Loading Optimized Model (Stripping legacy Keras arguments)...


2026-02-14 16:18:30.539313: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Success! Loaded 485577 features and stabilized the model.


In [1]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1. Create a stable background baseline (200 samples)
# We use 200 because it provides a statistically significant 'average' state
# for the model to compare individual patients against.
print("📉 Creating background baseline...")
background_indices = np.random.choice(X_train_sub.shape[0], 200, replace=False)
background_data = X_train_sub[background_indices]

# 2. Initialize the DeepExplainer
print("⏳ Initializing SHAP DeepExplainer...")
explainer = shap.DeepExplainer(final_model_v2, background_data)

# 3. Calculate SHAP values for 50 samples
# This will likely take 5-12 minutes on your CPU. 
print("🔍 Calculating SHAP values (Sit tight, this is heavy lifting)...")
# We explain 50 samples to get a robust distribution of feature importance
shap_values = explainer.shap_values(X_train_sub[:50])

# 4. Visualization for Subtype 1 (The minority class with 91% recall)
# This plot is the "Gold Standard" visual for a medical AI capstone.
plt.figure(figsize=(14, 10))
shap.summary_plot(
    shap_values[1], 
    X_train_sub[:50], 
    feature_names=cpg_names, 
    max_display=30, 
    plot_type="dot",
    show=False
)
plt.title("Top 30 Methylation Markers Driving Subtype Detection", fontsize=16)
plt.show()

# 5. Save the Results
# This creates a CSV you can reference in your final report
vals = np.abs(shap_values[1]).mean(0)
top_biomarkers = pd.DataFrame(list(zip(cpg_names, vals)), columns=['CpG_Site','Importance'])
top_biomarkers.sort_values(by=['Importance'], ascending=False, inplace=True)
top_biomarkers.to_csv('top_thyroid_biomarkers.csv', index=False)

print("\n🧬 Top 10 Biological Markers Identified:")
print(top_biomarkers.head(10))

📉 Creating background baseline...


NameError: name 'X_train_sub' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SpatialDropout1D
import shap
import pandas as pd
import matplotlib.pyplot as plt
import gc

# 1. Keras 3 Compatibility Wrapper (Fixes 'trainable' and 'noise_shape' errors)
class FixedSpatialDropout1D(SpatialDropout1D):
    def __init__(self, **kwargs):
        kwargs.pop('trainable', None)
        kwargs.pop('noise_shape', None)
        super().__init__(**kwargs)

# 2. Paths (Verified from your system)
DATA_PATH = '/home/Capstone_Team78/Dataset/Datasets/UnfilteredSubtypeData.npy'
MODEL_PATH = '/home/Capstone_Team78/Dataset/Models/thyroid-subtype-optimized.h5'

# 3. Memory-Safe Loading
print("📂 Loading Model and Feature Names...")
archive = np.load(DATA_PATH, allow_pickle=True)
cpg_names = archive['feature_names'].tolist()
custom_objects = {'SpatialDropout1D': FixedSpatialDropout1D}

# Loading the 92% accuracy model for inference
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)

# 4. Load Genomic Features
X_full = archive['X'].astype('float32')
print(f"✅ Loaded {X_full.shape[1]} features into 128GB RAM.")

# 5. Lean Background Baseline
# 50 samples is the "Sweet Spot" for i9 CPUs to stay fast and memory-stable
bg_indices = np.random.choice(X_full.shape[0], 50, replace=False)
background_data = X_full[bg_indices]

# 6. Initialize Explainer
print("⏳ Initializing DeepExplainer (using your 32 CPU threads)...")
explainer = shap.DeepExplainer(model, background_data)

# 7. Batch-Processing (10 samples per batch to prevent RAM spikes)
print("🔍 Calculating SHAP values in batches of 10...")
all_shap_values = []
num_to_explain = 30 # We explain 30 samples for a statistically significant summary

for i in range(0, num_to_explain, 10):
    print(f"   - Processing batch {i//10 + 1}/3...")
    batch = X_full[i:i+10]
    batch_shap = explainer.shap_values(batch)
    all_shap_values.append(batch_shap)
    gc.collect() # Manually clearing memory after each batch

# 8. Consolidate for Subtype 1 (91% recall class)
final_shap_values = np.concatenate([v[1] for v in all_shap_values], axis=0)
X_display = X_full[:num_to_explain]

# 9. Visualization
print("📊 Generating Clinical Summary Plot...")
plt.figure(figsize=(14, 10))
shap.summary_plot(
    final_shap_values, 
    X_display, 
    feature_names=cpg_names, 
    max_display=20, 
    plot_type="dot",
    show=False
)
plt.title("Top 20 Methylation Biomarkers for Thyroid Subtype", fontsize=16)
plt.savefig('thyroid_shap_summary.png', bbox_inches='tight')
plt.show()

# 10. Save Top Biomarkers to CSV for Capstone Report
vals = np.abs(final_shap_values).mean(0)
top_biomarkers = pd.DataFrame(list(zip(cpg_names, vals)), columns=['CpG_Site','SHAP_Importance'])
top_biomarkers.sort_values(by=['SHAP_Importance'], ascending=False, inplace=True)
top_biomarkers.to_csv('top_thyroid_biomarkers.csv', index=False)

print("\n🧬 SUCCESS: Top 10 Biological Markers Saved to CSV!")
print(top_biomarkers.head(10))

2026-02-14 16:23:21.704969: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-14 16:23:21.712173: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 16:23:21.984969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-14 16:23:22.875894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

📂 Loading Model and Feature Names...


/home/Capstone_Team78/venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
2026-02-14 16:23:24.225717: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Loaded 485577 features into 128GB RAM.
⏳ Initializing DeepExplainer (using your 32 CPU threads)...


/home/Capstone_Team78/venv/lib/python3.12/site-packages/shap/explainers/_deep/deep_tf.py:94: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn(


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SpatialDropout1D
import shap
import pandas as pd
import matplotlib.pyplot as plt
import gc

# 1. Keras 3 Compatibility Wrapper
class FixedSpatialDropout1D(SpatialDropout1D):
    def __init__(self, **kwargs):
        kwargs.pop('trainable', None)
        kwargs.pop('noise_shape', None)
        super().__init__(**kwargs)

# 2. Paths
DATA_PATH = '/home/Capstone_Team78/Dataset/Datasets/UnfilteredSubtypeData.npy'
MODEL_PATH = '/home/Capstone_Team78/Dataset/Models/thyroid-subtype-optimized.h5'

# 3. Load Data & Filter Features (Crucial Step)
print("Loading and Filtering Features...")
archive = np.load(DATA_PATH, allow_pickle=True)
X_raw = archive['X'].astype('float32')
cpg_names_raw = np.array(archive['feature_names'])

# Keep only the top 50,000 features with the highest variance
# These are the sites that show the most difference between cancer subtypes
variances = np.var(X_raw, axis=0)
top_indices = np.argsort(variances)[-50000:] 

X_filtered = X_raw[:, top_indices]
cpg_names_filtered = cpg_names_raw[top_indices].tolist()

# 4. Reload Model and Adapt Architecture
# Note: We must slice the model's weights to match the 50k inputs
print("Adapting Model for 50k features...")
custom_objects = {'SpatialDropout1D': FixedSpatialDropout1D}
full_model = tf.keras.models.load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)

# 5. Initialize SHAP with Filtered Data
print("⏳ Initializing DeepExplainer (Reduced to 50,000 features)...")
bg_indices = np.random.choice(X_filtered.shape[0], 50, replace=False)
background_data = X_filtered[bg_indices]

# We use a custom function to handle the feature slicing for the explainer
def model_predict_filtered(data):
    # This pads the 50k features back to 485k with zeros so the model can read it
    full_input = np.zeros((data.shape[0], 485577), dtype='float32')
    full_input[:, top_indices] = data
    return full_model.predict(full_input)

# Using KernelExplainer here as it is much more memory-stable than DeepExplainer
explainer = shap.KernelExplainer(model_predict_filtered, background_data)

# 6. Calculate SHAP for 10 samples (Very Safe)
print("🔍 Calculating SHAP values...")
shap_values = explainer.shap_values(X_filtered[:10], nsamples=100)

# 7. Visualization
plt.figure(figsize=(14, 10))
shap.summary_plot(shap_values[1], X_filtered[:10], feature_names=cpg_names_filtered, max_display=20)
plt.show()

2026-02-14 16:52:47.189442: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-14 16:52:47.196989: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 16:52:47.458820: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-14 16:52:48.316480: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

Loading and Filtering Features...
Adapting Model for 50k features...


/home/Capstone_Team78/venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
2026-02-14 16:52:50.503826: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


⏳ Initializing DeepExplainer (Reduced to 50,000 features)...
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step
🔍 Calculating SHAP values...


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 360s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 361s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

<Figure size 1400x1000 with 0 Axes>